In [1]:
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from matplotlib.ticker import MaxNLocator

import torchmetrics
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

from multiprocessing import cpu_count

In [2]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

rcParams['figure.figsize'] = 16, 10

tqdm.pandas()

In [3]:
pl.seed_everything(42)

Seed set to 42


42

In [4]:
X_train = pd.read_csv('./career-con-2019/X_train.csv')
y_train = pd.read_csv('./career-con-2019/y_train.csv')

In [5]:
y_train.head()

,series_id,group_id,surface
0,0,13,fine_concrete
1,1,31,concrete
2,2,20,concrete
3,3,31,concrete
4,4,22,soft_tiles


In [6]:
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(y_train.surface)
encoded_labels[:5]

array([2, 1, 1, 1, 6])

In [7]:
label_encoder.classes_

array(['carpet', 'concrete', 'fine_concrete', 'hard_tiles',
       'hard_tiles_large_space', 'soft_pvc', 'soft_tiles', 'tiled',
       'wood'], dtype=object)

In [8]:
y_train['label'] = encoded_labels
y_train.head()

,series_id,group_id,surface,label
0,0,13,fine_concrete,2
1,1,31,concrete,1
2,2,20,concrete,1
3,3,31,concrete,1
4,4,22,soft_tiles,6


In [9]:
FEATURE_COLUMNS = X_train.columns.tolist()[3:]
FEATURE_COLUMNS

['orientation_X',
 'orientation_Y',
 'orientation_Z',
 'orientation_W',
 'angular_velocity_X',
 'angular_velocity_Y',
 'angular_velocity_Z',
 'linear_acceleration_X',
 'linear_acceleration_Y',
 'linear_acceleration_Z']

In [10]:
LABEL_COLUMNS = y_train.columns.tolist()

In [11]:
# checking to see if each of the examples has a label in the y_train

(X_train.series_id.value_counts() == 128).sum() == len(y_train)

True

In [12]:
sequences = []

for series_id, group in X_train.groupby('series_id'):
  sequence_features = group[FEATURE_COLUMNS]

  label = y_train[y_train.series_id == series_id].iloc[0].label

  sequences.append((sequence_features, label))

In [13]:
train_sequences, test_sequences = train_test_split(sequences,test_size=0.2)

In [14]:
len(train_sequences), len(test_sequences)

(3048, 762)

In [15]:
class SurfaceDataset(Dataset):

  def __init__(self,sequences):
    self.sequences = sequences

  def __len__(self):
    return len(self.sequences)

  def __getitem__(self,idx):
    sequence, label = self.sequences[idx]
    sequence_tensor = torch.Tensor(sequence.to_numpy())
    # Ensure label is a tensor with the same dimensions as sequence
    label_tensor = torch.Tensor([label]).long()
    return dict(
        sequence = sequence_tensor,
        labels = label_tensor
    )

In [16]:
class SurfaceDataModule(pl.LightningDataModule):

  def __init__(self, train_sequences, test_sequences, batch_size=8):
    super().__init__()
    self.batch_size = batch_size
    self.train_sequences = train_sequences
    self.test_sequences = test_sequences

  def setup(self, stage=None):
    self.train_dataset = SurfaceDataset(self.train_sequences)
    self.test_dataset = SurfaceDataset(self.test_sequences)

  def train_dataloader(self):
    return DataLoader(
      self.train_dataset,
      batch_size=self.batch_size,
      shuffle=True,
      num_workers=cpu_count()
    )

  def val_dataloader(self):
    return DataLoader(
      self.test_dataset,
      batch_size=self.batch_size,
      shuffle=False,
      num_workers=cpu_count()
    )

  def test_dataloader(self):
    return DataLoader(
      self.test_dataset,
      batch_size=self.batch_size,
      shuffle=False,
      num_workers=cpu_count()
    )

In [17]:
N_EPOCHS = 10
BATCH_SIZE = 64

data_module = SurfaceDataModule(
  train_sequences,
  test_sequences,
  batch_size=BATCH_SIZE
)

In [18]:
class SequenceModel(nn.Module):
  def __init__(self,n_features, n_classes, n_hidden=256, n_layers=3):
    super().__init__()
    
    self.lstm = nn.LSTM(
        input_size=n_features,
        hidden_size=n_hidden,
        num_layers=n_layers,
        batch_first=True,
        dropout=0.75
    )

    self.classifier = nn.Linear(n_hidden,n_classes)

  def forward(self,x):
    self.lstm.flatten_parameters()
    _,(hidden,_) = self.lstm(x)

    out = hidden[-1]
    return self.classifier(out)


In [54]:
class SurfacePredictor(pl.LightningModule):

  def __init__(self,n_features:int, n_classes: int):
    super().__init__()
    self.model = SequenceModel(n_features, n_classes)
    self.criterion = nn.CrossEntropyLoss()
    self.n_classes = n_classes

  def forward(self, x, labels=None):
    output = self.model(x)
    loss = 0
    if labels is not None:
        loss = self.criterion(output, labels)
    return loss, output

  def training_step(self, batch, batch_idx):
    sequences = batch["sequence"]
    labels = batch["labels"].squeeze()
    loss, outputs = self(sequences, labels)
    predictions = torch.argmax(outputs,dim=1)
    step_accuracy = torchmetrics.functional.accuracy(predictions, labels, task="multiclass", num_classes=self.n_classes)

    self.log("train_loss", loss, prog_bar=True, logger=True)
    self.log("train_accuracy", step_accuracy, prog_bar=True, logger=True)
    return {"loss": loss, "accuracy": step_accuracy}

  def validation_step(self, batch, batch_idx):
    sequences = batch["sequence"]
    labels = batch["labels"].squeeze()
    loss, outputs = self(sequences, labels)
    predictions = torch.argmax(outputs,dim=1)
    step_accuracy = torchmetrics.functional.accuracy(predictions, labels, task="multiclass", num_classes=self.n_classes)

    self.log("val_loss", loss, prog_bar=True, logger=True)
    self.log("val_accuracy", step_accuracy, prog_bar=True, logger=True)
    return {"loss": loss, "accuracy": step_accuracy}

  def test_step(self, batch, batch_idx):
    
    sequences = batch["sequence"]
    labels = batch["labels"].squeeze()
    loss, outputs = self(sequences, labels)
    predictions = torch.argmax(outputs,dim=1)
    step_accuracy = torchmetrics.functional.accuracy(predictions, labels, task="multiclass", num_classes=self.n_classes)

    self.log("test_loss", loss, prog_bar=True, logger=True)
    self.log("test_accuracy", step_accuracy, prog_bar=True, logger=True)
    return {"loss": loss, "accuracy": step_accuracy}

  
  def configure_optimizers(self):
    return optim.Adam(self.parameters(), lr=0.0001)

In [55]:
model = SurfacePredictor(n_features=len(FEATURE_COLUMNS),n_classes=len(label_encoder.classes_))

In [56]:
%load_ext tensorboard
%tensorboard --logdir ./lightning_logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 14216), started 0:38:50 ago. (Use '!kill 14216' to kill it.)

In [57]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints",
    filename="best-checkpoint",
    save_top_k=1,
    verbose=True,
    monitor="val_loss",
    mode="min"
)

early_stopping_callback = EarlyStopping(monitor='val_loss', patience=2)

trainer = pl.Trainer(
    callbacks=[early_stopping_callback, checkpoint_callback],
    max_epochs=N_EPOCHS,
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [58]:
trainer.fit(model, data_module)

/home/nick/anaconda3/envs/wearables/lib/python3.12/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:653: Checkpoint directory /home/nick/Documents/ss24/ds4w-user-identification/checkpoints exists and is not empty.

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | SequenceModel    | 1.3 M 
1 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
1.3 M     Trainable params
0         Non-trainable params
1.3 M     Total params
5.318     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/nick/anaconda3/envs/wearables/lib/python3.12/site-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (48) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Epoch 7, global step 384: 'val_loss' reached 1.92675 (best 1.92675), saving model to '/home/nick/Documents/ss24/ds4w-user-identification/checkpoints/best-checkpoint-v4.ckpt' as top 1


Validation: |          | 0/? [00:00<?, ?it/s]

Epoch 8, global step 432: 'val_loss' reached 1.87658 (best 1.87658), saving model to '/home/nick/Documents/ss24/ds4w-user-identification/checkpoints/best-checkpoint-v4.ckpt' as top 1


Validation: |          | 0/? [00:00<?, ?it/s]

Epoch 9, global step 480: 'val_loss' was not in top 1
`Trainer.fit` stopped: `max_epochs=10` reached.


In [59]:
trainer.test(datamodule=data_module)

/home/nick/anaconda3/envs/wearables/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:145: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
Restoring states from the checkpoint path at /home/nick/Documents/ss24/ds4w-user-identification/checkpoints/best-checkpoint-v4.ckpt
Loaded model weights from the checkpoint at /home/nick/Documents/ss24/ds4w-user-identification/checkpoints/best-checkpoint-v4.ckpt


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_accuracy         0.29658791422843933
        test_loss           1.8765772581100464
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 1.8765772581100464, 'test_accuracy': 0.29658791422843933}]

In [60]:
trained_model = SurfacePredictor.load_from_checkpoint(
  trainer.checkpoint_callback.best_model_path,
  n_features=len(FEATURE_COLUMNS),
  n_classes=len(label_encoder.classes_)
)
trained_model.eval()
trained_model.freeze()

In [61]:
from tqdm.notebook import tqdm 
test_dataset = SurfaceDataset(test_sequences)

predictions = []
labels = []

for item in tqdm(test_dataset):
  sequence = item["sequence"]
  label = item["labels"]

  _, output = trained_model(sequence.unsqueeze(dim=0))
  prediction = torch.argmax(output, dim=1)
  predictions.append(prediction.item())
  labels.append(label.item())

  0%|          | 0/762 [00:00<?, ?it/s]

In [62]:
print(
    classification_report(labels, predictions, target_names=label_encoder.classes_ )
)

                        precision    recall  f1-score   support

                carpet       0.00      0.00      0.00        47
              concrete       0.25      0.64      0.36       172
         fine_concrete       0.00      0.00      0.00        65
            hard_tiles       0.00      0.00      0.00         5
hard_tiles_large_space       0.35      0.68      0.46        63
              soft_pvc       0.38      0.50      0.44       141
            soft_tiles       0.00      0.00      0.00        63
                 tiled       0.00      0.00      0.00        89
                  wood       0.29      0.02      0.03       117

              accuracy                           0.30       762
             macro avg       0.14      0.20      0.14       762
          weighted avg       0.20      0.30      0.20       762



/home/nick/anaconda3/envs/wearables/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/nick/anaconda3/envs/wearables/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/nick/anaconda3/envs/wearables/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capita

In [63]:
def show_confusion_matrix(confusion_matrix):
  hmap = sns.heatmap(confusion_matrix, annot=True, fmt="d", cmap="Blues")
  hmap.yaxis.set_ticklabels(hmap.yaxis.get_ticklabels(), rotation=0, ha='right')
  hmap.xaxis.set_ticklabels(hmap.xaxis.get_ticklabels(), rotation=30, ha='right')
  plt.ylabel('True surface')
  plt.xlabel('Predicted surface')

In [64]:
cm = confusion_matrix(labels, predictions)
df_cm = pd.DataFrame(
    cm, index=label_encoder.classes_, columns=label_encoder.classes_
)
df_cm

,carpet,concrete,fine_concrete,hard_tiles,hard_tiles_large_space,soft_pvc,soft_tiles,tiled,wood
carpet,0,44,0,0,3,0,0,0,0
concrete,0,110,0,0,36,23,0,0,3
fine_concrete,0,50,0,0,6,8,0,0,1
hard_tiles,0,0,0,0,0,5,0,0,0
hard_tiles_large_space,0,14,0,0,43,6,0,0,0
soft_pvc,0,63,0,0,6,71,0,0,1
soft_tiles,0,20,0,0,9,34,0,0,0
tiled,0,65,0,0,7,17,0,0,0
wood,0,81,0,0,13,21,0,0,2
